In [236]:
# Import dependencies
import pandas as pd
import os
import glob
import numpy as np
import datetime 

In [237]:
# Use glob to get all the supply data csv files in the current working directory
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

In [238]:
# Create empty list
dataframes_list = []

In [239]:
# Append datasets to the list 
for i in csv_files:
    temp_df = pd.read_csv(i)
    dataframes_list.append(temp_df)

In [240]:
# Check dataframes_list length for all suppy data CSV files
len(dataframes_list)

1362

In [241]:
# Create empty DataFrame for all supply data
supply_df = pd.DataFrame()

In [249]:
# Transform each DataFrame in list
for df in dataframes_list:    
    # Create date variable
    date = df.columns[0]

    # Reset index to energy sources and transpose
    df_transposed = df.set_index(date).T
    
    # Create date column
    df_transposed["Date"] = date
    
    # Create time column 
    df_transposed["Time"] = df_transposed.index

    # Reset index to start from 0
    df_transposed.reset_index(drop=True, inplace=True)
    
    # Cast 'Time' column to string and append seconds
    df_transposed['Time'] = df_transposed['Time'].apply(str) + ':00'
    df_transposed.iloc[0:120, 9] = "0" + df_transposed.iloc[0:120, 9]
    
    # Add 'Date' and 'Time' columns together
    df_transposed['DateTime'] = df_transposed['Date'] + ' ' + df_transposed['Time']

    try:
        # Covert 'DateTime' column to datetime 
        df_transposed['DateTime'] =  pd.to_datetime(df_transposed['DateTime'])
    except:
        continue
        
    try:
        # Create 'Unix Timestamp' column
        df_transposed['Unix Timestamp'] = df_transposed.DateTime.apply(lambda x : (x-datetime.datetime(1970,1,1)).total_seconds())
    except:
        continue
    
# Append to master DataFrame with all supply data
supply_df = supply_df.append(dataframes_list, ignore_index = True) 
supply_df.head()
    
# Export supply_df to CSV
    

,4/10/2018,0:00,0:05,0:10,0:15,0:20,0:25,0:30,0:35,0:40,...,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Unix Timestamp
0,Renewables,2005.0,2005.0,2048.0,1979.0,1980.0,1992.0,2003.0,1994.0,1990.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,Natural gas,8736.0,8722.0,8739.0,8696.0,8605.0,8550.0,8442.0,8365.0,8570.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,Large hydro,3428.0,3425.0,3464.0,3470.0,3454.0,3421.0,3419.0,3385.0,3090.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,Imports,7108.0,7029.0,6873.0,6744.0,6757.0,6720.0,6718.0,6727.0,6880.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,Batteries,-8.0,1.0,6.0,-14.0,-38.0,-39.0,-36.0,-35.0,-4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
